In [1]:
### 1. Importing Libraries and Data Preprocessing
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from glob import glob
from sklearn.model_selection import train_test_split
from skimage.transform import resize
from keras.utils import to_categorical
from keras.applications.vgg16 import preprocess_input, VGG16
from keras.models import Sequential
from keras.layers import Dense, InputLayer, Flatten
from keras.optimizers import Adam  # Using Adam optimizer

### 2. Load and Preprocess the Data
data = pd.read_csv('data/emergency_classification.csv')
X = []
for img_name in data.image_names:
    img = plt.imread('data/images/' + img_name)
    img = resize(img, (224, 224), anti_aliasing=True, mode='reflect')  # Resize images
    X.append(img)
X = np.array(X)
X = preprocess_input(X)  # Preprocess the data
y = to_categorical(data.emergency_or_not.values)

### 3. Split the Dataset
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.3, random_state=42)

### 4. Load the Pretrained VGG16 Model
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
base_model.summary()  # Optional: to view model structure

### 5. Fine-Tune the Model
# Extract features
X_train_features = base_model.predict(X_train)
X_valid_features = base_model.predict(X_valid)

# Flatten extracted features
X_train_features_flat = X_train_features.reshape(X_train_features.shape[0], -1)
X_valid_features_flat = X_valid_features.reshape(X_valid_features.shape[0], -1)

# Normalize features
max_val = X_train_features_flat.max()
X_train_features_flat /= max_val
X_valid_features_flat /= max_val

# Define and Compile Model
model = Sequential([
    InputLayer(input_shape=(X_train_features_flat.shape[1],)),
    Dense(1024, activation='relu'),
    Dense(2, activation='softmax')
])
model.compile(optimizer=Adam(learning_rate=1e-4), loss='categorical_crossentropy', metrics=['accuracy'])

# Train the Model
model.fit(X_train_features_flat, y_train, epochs=100, validation_data=(X_valid_features_flat, y_valid))

# Evaluate the Model
predictions = model.predict(X_valid_features_flat)
predicted_classes = np.argmax(predictions, axis=1)
true_classes = np.argmax(y_valid, axis=1)
accuracy = np.mean(predicted_classes == true_classes)
print(f'Validation Accuracy: {accuracy * 100:.2f}%')


2024-02-05 10:56:25.066695: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M1
2024-02-05 10:56:25.066717: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 16.00 GB
2024-02-05 10:56:25.066721: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 5.33 GB
2024-02-05 10:56:25.066997: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:306] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2024-02-05 10:56:25.067243: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:272] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


Model: "vgg16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0     

2024-02-05 10:56:29.016404: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:117] Plugin optimizer for device_type GPU is enabled.


23/23 [==============================] - 9s 412ms/step


Epoch 1/100


2024-02-05 10:57:01.902138: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:961] model_pruner failed: INVALID_ARGUMENT: Graph does not contain terminal node Adam/AssignAddVariableOp.


52/52 [==============================] - 3s 46ms/step - loss: 0.6820 - accuracy: 0.5784 - val_loss: 0.6868 - val_accuracy: 0.5609
Epoch 2/100
52/52 [==============================] - 2s 39ms/step - loss: 0.6778 - accuracy: 0.5863 - val_loss: 0.6847 - val_accuracy: 0.5609
Epoch 3/100
52/52 [==============================] - 2s 39ms/step - loss: 0.6772 - accuracy: 0.5863 - val_loss: 0.6833 - val_accuracy: 0.5609
Epoch 4/100
52/52 [==============================] - 2s 40ms/step - loss: 0.6758 - accuracy: 0.5863 - val_loss: 0.6838 - val_accuracy: 0.5609
Epoch 5/100
52/52 [==============================] - 2s 39ms/step - loss: 0.6753 - accuracy: 0.5863 - val_loss: 0.6842 - val_accuracy: 0.5609
Epoch 6/100
52/52 [==============================] - 2s 39ms/step - loss: 0.6733 - accuracy: 0.5863 - val_loss: 0.6837 - val_accuracy: 0.5609
Epoch 7/100
52/52 [==============================] - 2s 38ms/step - loss: 0.6750 - accuracy: 0.5863 - val_loss: 0.6815 - val_accuracy: 0.5609
Epoch 8/100
52/52 